In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from cyvcf2 import VCF

In [ ]:
project_folder = "diygenomics-projects"
sub_category = "DATA"
work_bucket = "genomes"

base_file_name = 'GCF_000001405.40'
input_file = f'{base_file_name}.gz'
output_file = f'{base_file_name}.db'

In [ ]:
data_path = '/Volumes/cold_storage/data' # os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, project_folder, sub_category, work_bucket, *args)

In [ ]:
conn = sqlite3.connect(file_path(output_file))

In [ ]:
cur = conn.cursor()

cur.execute("""
    CREATE TABLE vcf_data (
        RSID TEXT,
        PSEUDOGENEINFO TEXT,
        GENEINFO TEXT
    )
""")

In [ ]:
vcf = VCF(file_path(input_file))

In [ ]:
counter = 0
for record in vcf:
    cur.execute("""
        INSERT INTO vcf_data (RSID, PSEUDOGENEINFO, GENEINFO)
        VALUES (?, ?, ?)
    """, (record.ID, record.INFO.get('PSEUDOGENEINFO'), record.INFO.get('GENEINFO')))
    counter += 1
    if counter % 10_000_000 == 0:
        print(f'Processed {counter} records')

In [ ]:
conn.commit()
cur.close()
conn.close()